# ***Day 3 — Build a Text Generator (Mini ChatGPT)***

🧩 What You Are Building Today

* A Mini ChatGPT / Text Generator that:

1. Takes user input (prompt)

2. Sends it to a Transformer model

3. Generates AI text as output



In [ ]:
!pip install transformers --quiet

from transformers import pipeline
generator = pipeline("text-generation", model="distilgpt2")



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def generate_text(prompt):
    result = generator(
        prompt,
        max_new_tokens=80,
        temperature=0.9,
        top_p=0.95,
        num_return_sequences=1
    )
    return result[0]["generated_text"]

prompt = "Artificial Intelligence will change the world by"
output = generate_text(prompt)

print("AI Output:\n")
print(output)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AI Output:

Artificial Intelligence will change the world by moving to a new approach to it. The technology is being developed in the cloud, one that was first introduced by Google as a way for companies to do business with their data.







The company is now working on a "cloud platform," which will allow customers to store their data in your cloud.

The cloud platform is already using Google Drive as a way


In [ ]:
user_prompt = input("Enter your prompt: ")
response = generate_text(user_prompt)

print("\n🤖 AI Response:\n")
print(response)


Enter your prompt: life


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🤖 AI Response:

life A very popular, non-commercial, non-commercial, non-commercial, non-commercial video game, which takes the player to the beach.




















































In [ ]:
def generate_multiple(prompt):
    results = generator(
        prompt,
        max_new_tokens=60,
        temperature=1.0 ,
        num_return_sequences=3
    )
    for i, r in enumerate(results):
        print(f"\n--- Response {i+1} ---")
        print(r["generated_text"])

generate_multiple("i love you ")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- Response 1 ---
i love you !! I know the time has come (and we didn't see some). So I just want to be honest with you…this game for the first time! I love it so much. :D
I'm looking forward to the upcoming release but I have no doubt that, from the start anyway

--- Response 2 ---
i love you !!! LOVE you!!! THANKS TO MY KIDS TO ALL OF YOU AND TO THE KIDS. MOST LIKE YOU.
LIVE TWO BODY!

--- Response 3 ---
i love you !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



# 1️⃣ How **Text Generators Work Internally** (CORE IDEA)

A text generator **does NOT think**.
It **predicts the next word** again and again.

### Simple example:

Sentence:

> I love ice

Model asks:
👉 “What word usually comes after `I love ice`?”

From training data, it knows:

* cream (70%)
* skating (10%)
* coffee (5%)
* coding (0.1%)

It **chooses one**, adds it:

> I love ice cream

Then repeats:

> “What comes after `I love ice cream`?”

This repeats **token by token** until stopping.

📌 **That’s it.**
No brain. No emotions. Just probability.

---

# 2️⃣ How **Prompts Flow Into a Model**

Your prompt is just **input text**.

### What really happens:

```
Your Prompt (text)
   ↓
Tokenizer (text → numbers)
   ↓
Transformer Model (math + probabilities)
   ↓
Generated Tokens (numbers)
   ↓
Detokenizer (numbers → text)
```

### Example:

```python
prompt = "AI will replace"
```

Internally becomes:

```
[1012, 4532, 9981]   ← numbers (tokens)
```

Model predicts next token:

```
→ [work]
→ [jobs]
→ [humans]
```

📌 Prompt = **starting context**, not instructions.

---

# 3️⃣ What is a **Transformer** (IMPORTANT)

A **Transformer** is the architecture that:

* Understands **context**
* Looks at **all words together**, not one by one
* Uses **Attention**

### Attention means:

> “Which words matter most right now?”

Example:

> **The animal didn’t cross the road because it was tired**

What is **it**?

* animal ✅
* road ❌

Transformer figures this using **attention weights**.

📌 That’s why transformers are powerful.

---

# 4️⃣ How Output is Controlled (VERY IMPORTANT)

Now your confusion parts 👇
These are NOT random magic values.

---

## 🔥 `temperature` — Creativity Control

Controls **how risky the word choice is**.

### Low temperature (0.2 – 0.5)

* Safe
* Repetitive
* Boring
* Most probable word chosen

### High temperature (0.9 – 1.2)

* Creative
* Random
* Emotional
* Sometimes nonsense

📌 Example:

```python
temperature = 0.3  # strict, logical
temperature = 1.0  # creative
```

---

## 🔥 `top_p` — Word Selection Filter

Also called **nucleus sampling**.

Model has MANY word choices.
`top_p` says:

> “Only consider words that make up X% probability”

### Example:

If `top_p = 0.9`
→ Ignore weird low-probability words

📌 Helps avoid nonsense while keeping creativity.

---

## 🔥 `max_new_tokens` — Length Control

This is **how many words/tokens AI can add**.

```python
max_new_tokens = 50
```

Means:

> AI can generate **up to 50 tokens AFTER your prompt**

If prompt is long, it doesn’t matter.

📌 This is better than `max_length` (old method).

---

# 5️⃣ Why Your Output Was WEIRD (IMPORTANT)

You saw:

> OpenOffice nonsense text

WHY?

Because:

* `distilgpt2` is **small**
* Trained on **old + noisy data**
* High randomness
* No safety filters

📌 That’s expected. Not your fault.

Bigger models = better output.

---

# 6️⃣ How a **Basic Chatbot Is Structured**

A chatbot is **NOT special**.

It is just:

```python
conversation = ""

while True:
    user_input = input("You: ")
    conversation += "User: " + user_input + "\nAI:"
    response = generator(conversation)
    print(response)
```

📌 Chatbot =
**Text generator + memory (conversation history)**

That’s ALL.

---

